In [2]:
from typing import Callable, List, Dict, Tuple, Union

# Types
Vector = List[float]
Node = str
Relation = str
Dimension = str
Target = Tuple[Union[Node, Relation], Dimension]

# Structure du SEAF
class SEAF:
    def __init__(self,
                 nodes: List[Node],
                 node_dims: List[Dimension],
                 relations: List[Relation],
                 rel_dims: List[Dimension],
                 val: Dict[Union[Node, Relation], Vector],
                 source: Dict[Relation, Node],
                 target: Dict[Relation, Target],
                 polarity: Dict[Relation, int]):
        self.nodes = nodes
        self.node_dims = node_dims
        self.relations = relations
        self.rel_dims = rel_dims
        self.val = val
        self.source = source
        self.target = target
        self.polarity = polarity

# Sémantique graduelle avec point fixe
def evaluate_gradual_semantics_fixedpoint(
    seaf: SEAF,
    f: Callable[[float, float], float],
    g: Callable[[List[float]], float],
    h: Callable[[float, float], float],
    q_n: Callable[[Vector], float],
    q_r: Callable[[Vector], float],
    epsilon: float = 1e-4,
    max_iter: int = 100,
    debug: bool = False
) -> Dict[Union[Node, Relation], Vector]:

    current_vals = {k: v[:] for k, v in seaf.val.items()}

    for iteration in range(max_iter):
        if debug:
            print(f"\n=== Iteration {iteration + 1} ===")
        new_vals = {}

        for x in seaf.nodes + seaf.relations:
            dims = seaf.node_dims if x in seaf.nodes else seaf.rel_dims
            updated = []
            print(x)
            for i, d in enumerate(dims):
                g_list = []
                for r in seaf.relations:
                    tgt, dim = seaf.target[r]
                    if tgt == x and dim == d:
                        rel_val = current_vals[r]
                        src_val = current_vals[seaf.source[r]]
                        h_agg = h(q_r(rel_val), q_n(src_val)) * (seaf.polarity[r])
                        g_list.append(h_agg)
                        if debug:
                            rounded_rel = [round(v, 3) for v in rel_val]
                            rounded_src = [round(v, 3) for v in src_val]
                            print(f"[{x}.{d}] <- {r}: h(q_r({rounded_rel}) = {q_r(rel_val):.3f}, q_n({rounded_src}) = {q_n(src_val):.3f}) * polarity {seaf.polarity[r]} = {h_agg:.3f}")
                g_agg = g(g_list) if g_list else 0.0
                new_value = f(seaf.val[x][i], g_agg)
                updated.append(new_value)
                if debug:
                    if len(g_list) > 0:
                        rounded_g_list = [round(v, 3) for v in g_list]
                        print(f"   g({rounded_g_list}) = {g_agg:.3f}  ---  f({seaf.val[x][i]:.3f}, {g_agg:.3f}) = {new_value:.3f}")
            new_vals[x] = updated

        max_delta = max(
            abs(new_vals[x][i] - current_vals[x][i])
            for x in current_vals
            for i in range(len(current_vals[x]))
        )
        current_vals = new_vals
        if max_delta < epsilon:
            if debug:
                print(f"\nConverged after {iteration + 1} iterations (max delta = {max_delta:.6f})")
            break

    return current_vals

# Fonctions d'évaluation
def f_att(x: float, y: float) -> float:
    return x / (1 - y)

def impact_pwr(x,y):
    return (max(0,x)*x**y) / (1 + max(0,x)*x**y)
def f_bipolar(x,E):
    return x + (1-x)*impact_pwr(E,1) - (x*impact_pwr(-E,1))

def g(g_list: List[float]) -> float:
    return sum(g_list) if g_list else 0.0

def h(rel_strength: float, source_strength: float) -> float:
    return rel_strength * source_strength

def q_n(vec: List[float]) -> float:
    return sum(vec) / len(vec) if vec else 0.0

def q_r(vec: List[float]) -> float:
    return sum(vec) / len(vec) if vec else 0.0

# Exemple de SEAF
nodes = ["n1", "n2","n3","n4"]
node_dims = ["d1", "d2", "d3"]
relations = ["r12", "r21", "r32","r4r1"]
rel_dims = ["dr1"]

val = {
    "n1": [0.9, 0.7, 0.4],
    "n2": [0.5, 0.5, 0.8],
    "n3": [0.3, 0.8, 0.6],
    "n4": [0.4, 0.8, 0.9],
    "r12": [0.2],  # n1 → n2.d3
    "r21": [0.8],  # n2 → n1.d1
    "r32": [0.5],   # n3 → n2.d3
    "r4r1": [0.6]   # n4 → n4.dr1
}

source = {
    "r12": "n1",
    "r21": "n2",
    "r32": "n3",
    "r4r1": "n3"
}

target = {
    "r12": ("n2", "d3"),
    "r21": ("n1", "d1"),
    "r32": ("n2", "d3"),
    "r4r1": ("r12", "dr1")
}

polarity = {
    "r12": -1,
    "r21": -1,
    "r32": 1,
    "r4r1": -1
}

seaf = SEAF(
    nodes=nodes,
    node_dims=node_dims,
    relations=relations,
    rel_dims=rel_dims,
    val=val,
    source=source,
    target=target,
    polarity=polarity
)

# Exécution avec debug
results = evaluate_gradual_semantics_fixedpoint(
    seaf=seaf,
    f=f_bipolar,
    g=g,
    h=h,
    q_n=q_n,
    q_r=q_r,
    epsilon=1e-4,
    max_iter=100,
    debug=True
)

# Affichage final
print("\n=== Final Values (Rounded to 0.001, with Aggregation) ===")
for key, vec in results.items():
    rounded = [round(v, 3) for v in vec]
    agg = round(q_n(vec), 3) if key in seaf.nodes else round(q_r(vec), 3)
    print(f"{key}: {rounded} → aggregated: {agg}")



=== Iteration 1 ===
n1
[n1.d1] <- r21: h(q_r([0.8]) = 0.800, q_n([0.5, 0.5, 0.8]) = 0.600) * polarity -1 = -0.480
   g([-0.48]) = -0.480  ---  f(0.900, -0.480) = 0.731
n2
[n2.d3] <- r12: h(q_r([0.2]) = 0.200, q_n([0.9, 0.7, 0.4]) = 0.667) * polarity -1 = -0.133
[n2.d3] <- r32: h(q_r([0.5]) = 0.500, q_n([0.3, 0.8, 0.6]) = 0.567) * polarity 1 = 0.283
   g([-0.133, 0.283]) = 0.150  ---  f(0.800, 0.150) = 0.804
n3
n4
r12
[r12.dr1] <- r4r1: h(q_r([0.6]) = 0.600, q_n([0.3, 0.8, 0.6]) = 0.567) * polarity -1 = -0.340
   g([-0.34]) = -0.340  ---  f(0.200, -0.340) = 0.179
r21
r32
r4r1

=== Iteration 2 ===
n1
[n1.d1] <- r21: h(q_r([0.8]) = 0.800, q_n([0.5, 0.5, 0.804]) = 0.601) * polarity -1 = -0.481
   g([-0.481]) = -0.481  ---  f(0.900, -0.481) = 0.731
n2
[n2.d3] <- r12: h(q_r([0.179]) = 0.179, q_n([0.731, 0.7, 0.4]) = 0.610) * polarity -1 = -0.109
[n2.d3] <- r32: h(q_r([0.5]) = 0.500, q_n([0.3, 0.8, 0.6]) = 0.567) * polarity 1 = 0.283
   g([-0.109, 0.283]) = 0.174  ---  f(0.800, 0.174) = 0.80